In [ ]:
!pip install lightgbm scikit-learn pandas numpy matplotlib
!pip install dnspython


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.11/bin/python3.11 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.11/bin/python3.11 -m pip install --upgrade pip


In [10]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score


In [11]:
import sys
print(sys.executable)


/opt/anaconda3/bin/python


In [12]:
!{sys.executable} -m pip install lightgbm
!{sys.executable} -m pip install pymongo



In [13]:
import lightgbm as lgb
from pymongo import MongoClient
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder


In [15]:
client = MongoClient("mongo_uri")
db = client["db_name"]
collection = db["coll_name"]
all_documents = collection.find({})
data_list = list(all_documents) 
df = pd.DataFrame(data_list)
print(df.head())


#cleaning from mongo 
df = df.drop(columns=["_id"], errors="ignore")

#Convert everything that can be a number into numeric (others stay as is)
df = df.apply(pd.to_numeric, errors="ignore")
#For yes/no, true/false, convert to 0/1
for col in df.columns:
    if df[col].dtype == "object":
       df[col] = df[col].astype(str).str.lower().map(
            {"yes": 1, "no": 0, "true": 1, "false": 0}
        ).fillna(df[col])
print(df.dtypes)




                        _id  group_id      source   age    income  \
0  68f1db744a63589ed705fc7e         1  Online Ads  56.0  166867.0   
1  68f1db744a63589ed705fc7f         1  Online Ads  28.0  132727.0   
2  68f1db744a63589ed705fc80         1  Online Ads  61.0       NaN   
3  68f1db744a63589ed705fc81         1  Online Ads  44.0  176730.0   
4  68f1db744a63589ed705fc82         1  Online Ads  20.0   89092.0   

   credit_score  previous_products  clicks  time_on_site  loan_balance  \
0         570.0                2.0     7.0      5.986585        6265.0   
1         672.0                NaN     7.0      7.080726        1685.0   
2           NaN                NaN     0.0      3.042422           NaN   
3         810.0                1.0    15.0      2.327713           NaN   
4           NaN                0.0     6.0      0.132650           NaN   

   amount_spent_on_call  clicked_landing_page  branch_wait_time  \
0                   NaN                   NaN               NaN   
1     

/var/folders/vh/yh6fpcsx7t54h13svtd49lb80000gn/T/ipykernel_13043/474846519.py:14: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors="ignore")


In [17]:
y = df['converted']
X = df.drop(columns=['converted','source'])
group = df['group_id'] 

print("Features columns before split:", X.columns.tolist())

Features columns before split: ['group_id', 'age', 'income', 'credit_score', 'previous_products', 'clicks', 'time_on_site', 'loan_balance', 'amount_spent_on_call', 'clicked_landing_page', 'branch_wait_time', 'referral_bonus', 'ad_impressions']


In [ ]:
# Train-test split (80-20, stratify not needed for ranking)
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    X, y, group, test_size=0.2, random_state=42
)

# LightGBM dataset with groups
train_data = lgb.Dataset(X_train.drop(columns=["group_id"]), label=y_train, group=group_train.value_counts().sort_index().values)
test_data = lgb.Dataset(X_test.drop(columns=["group_id"]), label=y_test, group=group_test.value_counts().sort_index().values)

# Ranking parameters
params = {
    "objective": "lambdarank",   # ranking objective
    "metric": "ndcg",
    "boosting_type": "gbdt",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "verbose": -1
}

# Train the model
model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=100)

# Predict relevance score
X_test_pred = model.predict(X_test.drop(columns=["group_id"],errors="ignore"))

ndcg = ndcg_score([y_test], [X_test_pred], k=5)
print(f"NDCG@5 Score: {ndcg:.4f}")

# Predict for the FULL dataset (all 10,000 rows)
all_pred = model.predict(X.drop(columns=["group_id"], errors="ignore"))

results = X.copy()  
results["actual_conversion"] = y.values        
results["relevance_score"]   = all_pred
results["rank_global"]       = results["relevance_score"].rank(ascending=False, method="first")

# Sort overall by relevance_score
results = results.sort_values(by=["relevance_score"], ascending=[False])

results.to_csv("results_new.csv", index=False)
print("Results saved to results_new.csv")


model.save_model("lightgbm_lead_ranking_model.txt")
print("Model saved as lightgbm_lead_ranking_model.txt")

!python retrain_with_feedback.py --push


NDCG@5 Score: 0.1696
Results saved to results_new.csv
Model saved as lightgbm_lead_ranking_model.txt


In [30]:
#send results.csv to mongodb
from datetime import datetime
from pymongo.errors import CollectionInvalid

MONGODB_URI = "mongo_uri"
DB_NAME     = "db_name"
CSV_PATH    = "results_new.csv"   
COLL_NAME   = "coll_name"


# Connect 
client = MongoClient(MONGODB_URI, uuidRepresentation="standard")
coll = client[DB_NAME][COLL_NAME]

#insert
df = pd.read_csv(CSV_PATH).replace({np.nan: None})
docs = df.to_dict(orient="records")

# replace previous run 
coll.delete_many({})

if docs:
    coll.insert_many(docs)
    print(f"Inserted {len(docs)} docs into {DB_NAME}.{COLL_NAME}")
else:
    print("results_new.csv is empty")


Inserted 10000 docs into leadgenerate.lead_results_new
